<u>The <span style="color:green">**GrSyLi**</span> Package for GAP: <span style="color:green">Gr</span>aphs with <span style="color:green">Sy</span>mmetries <span style="color:green">Li</span>brary</u>
-----------------------------------------------------------------

The <span style="color:green">**GrSyLi**</span> package contains various collections of connected graphs with interesting symmetry properties. This includes:

  - all cubic vertex-transitive graphs on up to 1280 vertices
  - all arc-transitive 2-valent digraphs on up to 1000 vertices
  - all ($G$-)half-arc-transitive 4-valent graphs on up to 1000 vertices
  - all arc-transitive 4-valent graphs on up to 640 vertices
  - all edge-transitive 4-valent graphs on up to 512 vertices


--------------------------

The <span style="color:green">**GrSyLi**</span> package is based on the Digraphs package for GAP. The Digraph package provides lots of functionality for the inspection, construction and modification of (di)graphs, as well as various I/O capabilities.

In [ ]:
LoadPackage("grsyli");;

In [ ]:
NrCubicVTGraphs(50);

In [ ]:
gamma:=CubicVTGraph(50,8);

In [ ]:
Size(AutomorphismGroup(gamma));

In [ ]:
Size(Stabilizer(AutomorphismGroup(gamma),1));

In [ ]:
chi:=CharacteristicPolynomial(gamma);

In [ ]:
Collected(Factors(chi));

In [ ]:
DigraphDegeneracy(gamma);

In [ ]:
IsOuterPlanarDigraph(gamma);

In [ ]:
IsHamiltonianDigraph(gamma);

In [ ]:
IsDigraphCore(gamma);

In [ ]:
CliqueNumber(gamma);

In [ ]:
Splash(DotSymmetricDigraph(gamma));

If the user wants to use graphs in <span style="color:purple">**Grape**</span> format, you can simply use the command`Graph`.

In [ ]:
NrCubicVTGraphs(10);

In [ ]:
gammas:=List(AllCubicVTGraphs(10),Graph);;

In [ ]:
List(gammas,ChromaticNumber);

In [ ]:
List(gammas,IsDistanceRegular);

In [ ]:
# Build the petersen graph as the Odd graph O_5
petersen:=Graph(SymmetricGroup(5),[[1,2]],OnSets,
                                    function(x,y) 
                                      return Intersection(x,y)=[]; 
                                    end);;

In [ ]:
List(gammas,gamma->IsIsomorphicGraph(gamma,petersen));

<u> Stored parameters in the <span style="color:green">**GrSyLi**</span> package</u>
------------------------------------------------

The <span style="color:green">**GrSyLi**</span> package has many precomputed attributes stored for each cubic vertex-transitive graph it contains.
This includes:
 - Girth
 - Diameter
 - Cayley-ness
 - Arc-transitivity
 - Being isomorphic to a **Split Praeger-Xu graph**

To access this information, you can load the graph with the optional argument `data` set to `true`, or use the `SetCubicVTGraphProps` function (slow).

In [ ]:
SmallCubicVTGraphsInfo(400);

In [ ]:
gamma:=CubicVTGraph(400,10,true);

In [ ]:
IsBipartiteDigraph(gamma);

In [ ]:
IsCayleyGraph(gamma);

In [ ]:
IsSplitPraegerXuGraph(gamma);

All functionality is documented in the manual, which can be found in the github repository.

Once the user builds the documentation locally (see manual), they can use the GAP help viewer to find documentation on specific functions or sections.

In [ ]:
?IsSplitPraegerXuGraph


<u>Example: The degree-girth problem</u>
----------------------------------------

Now let's investigate the degree-girth problem for cubic graphs.

 - The **degree-girth** problem: what is the *minimum* order of a $k$-regular graph with girth $g$?

The Moore bound gives an upper bound for the order of a graph in the degree-diameter problem, and there is also a lower bound for the degree-girth problem which we call the **girth Moore bound** $M(k,g)$.  

In [ ]:
# Moore bound by girth for cubic graphs
GirthMooreBound:=function(gamma)
  local girth;

  girth:=DigraphUndirectedGirth(gamma);
  if IsEvenInt(girth) then
    return 2*(2^(girth/2)-1);
  else
    return 1+3*(2^((girth-1)/2) - 1);
  fi;
end;;

In [ ]:
gexact_graphs:=[];; max:=500;;

for n in [4,6..max] do
  iter:=CubicVTGraphIterator(n,true);

  for gamma in iter do

    girth:=DigraphUndirectedGirth(gamma);
    gmoore:=GirthMooreBound(gamma);

    if DigraphNrVertices(gamma)=gmoore then
      Add(gexact_graphs,gamma);
    fi;
  od;
od;

Length(gexact_graphs);

The 5 graphs meeting the girth bound are well known examples of distance-regular graphs:

  - the **tetrahedron graph** $K_4$,
  - the **complete multipartite graph** $K_{3,3}$,
  - the **Petersen graph**,
  - the **Heawood graph**, and
  - the **Tutte 8-cage**.

The <span style="color:blue">**AGT**</span> package (<span style="color:blue">A</span>lgebraic <span style="color:blue">G</span>raph <span style="color:blue">T</span>heory) contains specific constructors for all but the Tutte 8-cage. It can be constructed as the point-line incidence graph of the smallest generalised quadrangle $W(2)$.

In [ ]:
LoadPackage("agt");;

IsIsomorphicGraph(TetrahedronGraph(),Graph(gexact_graphs[1]));

In [ ]:
IsIsomorphicGraph(CompleteMultipartiteGraph([3,3]),Graph(gexact_graphs[2]));

In [ ]:
IsIsomorphicGraph(petersen,Graph(gexact_graphs[3]));

In [ ]:
IsIsomorphicGraph(HeawoodGraph(),Graph(gexact_graphs[4]));

In [ ]:
IsIsomorphicGraph(AGT_IncidenceGraph(GeneralizedQuadrangleW(2)),Graph(gexact_graphs[5]));

A related parameter is the **excess** of a given $k$-regular graph $G$, defined as 

<center>$e(G)=|V(G)|-M(k,g)$,</center>

where $g$ is the girth of $G$.

In particular, we are interested in how large or small the excess of a graph can be, for given $k$ and $g$.

In [ ]:
excess_min:=List([1..max],x->infinity);;
excess_graphs:=List([1..max],x->[]);;

for n in [4,6..max] do
  iter:=CubicVTGraphIterator(n,true);
  
  for gamma in iter do

    girth:=DigraphUndirectedGirth(gamma);
    gbound:=GirthMooreBound(gamma);
    
    e:=DigraphNrVertices(gamma)-gbound;

    excess_min[girth]:=Minimum(excess_min[girth],e);

    if e=excess_min[girth] then
      Add(excess_graphs[girth],gamma);
    fi;
  od;
od;

In [ ]:
PositionsProperty(excess_min,x->x<>infinity);
Filtered(excess_min,x->x<>infinity);

The unique cubic vertex-transitive graph of order 272 and girth 13 is best known, showing that $202\leq n(3,13)\leq 272$.

Discovered by Hoare, shown to be smallest possible Cayley graph by Royle.



In [ ]:
Length(excess_graphs[13]); gamma:=excess_graphs[13][1];;

In [ ]:
DigraphNrVertices(gamma); IsCayleyGraph(gamma);

<u>Example: Cayley Neumaier graphs using the small groups library</u>
-------------------------------------------------------------

In this example, we find the smallest strictly Neumaier graph using the small groups library. The approach goes as follows: 

- iterate through small groups of size the desired order
- search for possible connection sets of a Cayley graph, which have a specific structure that gaurantees the existence of a regular clique
- check if such a connection set gives an erg
- build the graphs, check if they are srgs and reduce by isomorphism class

It can be shown that each strictly Neumaier graph on at most 16 vertices must:

- have 16 vertices
- be 9-regular
- have each edge being in exactly 4 triangles
- contain a 2-regular 4-clique

In [ ]:
IsInverseClosed:=function(L)
  return ForAll(L,x->x^-1 in L);
end;;

v:=16;; k:=9;; lambda:=4;; m:=2;; s:=4;; idx:=v/s;;
connection_sets:=[];;

In [ ]:
for G in AllSmallGroups(v) do

  # Finding subgroups of size s up to conjugacy
  reps:=List(ConjugacyClassesSubgroups(G),Representative);;
  reps:=Filtered(reps,H->Size(H)=s);;

  # Reducing by automorphism
  aut:=AutomorphismGroup(G);
  reps:=List(Orbits(aut,List(reps,Elements),OnSets),First);

  for H in reps do
    
    # Build all possible sets T                             | General                                            : (16,9,4) case

    cos:=CosetDecomposition(G,Group(H)){[2..idx]};;         # cosets of H, not including H                       : 3 cosets 
    pairs:=List(cos,x->Combinations(x,m));;                 # m-subsets of coset representatives from each coset : 6 each
    cart:=Cartesian(pairs);;                                # all possible combinations of m-subsets             : 6^3=216 elements
    cart:=Filtered(cart,x->IsInverseClosed(Union(x)));;     # keep only inverse-closed subsets
    Ts:=List(cart,Union);;

    for T in Ts do
      if ForAll(H{[2..s]},h->Length(Intersection(T,T*h))=lambda-s+2) and
         ForAll(T,g->Length(Intersection(T,T*g))=lambda-2*m+2) then
           Add(connection_sets,[G,Union(H{[2..s]},T)]);
      fi;
    od;
  od;
od;    

In [ ]:
gammas:=List(connection_sets,l->CayleyGraph(l[1],l[2]));;
gammas:=GraphIsomorphismClassRepresentatives(gammas);;
Length(gammas);

There are exactly 2 strongly regular graphs with these parameters, so we know we have found at least one strictly Neumaier graph.

To investigate further, we can use the <span style="color:blue">**AGT**</span> package.

In [ ]:
LoadPackage("agt");;
List(gammas,IsSRG);

In [ ]:
IsIsomorphicGraph(ComplementGraph(ShrikhandeGraph()),gammas[1]);

In [ ]:
IsIsomorphicGraph(BilinearFormsGraph(2,2,2),gammas[2]);                  # complement of L_2(4)

In [ ]:
IsERG(gammas[3]);

In [ ]:
CliqueNumber(gammas[3]);

In [ ]:
IsNG(gammas[3]);